# Attention is All You Need

Trabalho complementar ao seminário apresentado no Laboratório de Engenharia de Computação Científica (LECC), pertencente ao Programa de Engenharia de Sistemas e Computação (PESC) - UFRJ. 

Aqui temos como objetivo destrinchar e entender por completo o modelo de Encoder-Decoder que introduziu a arquitetura de Transformers na literatura, originalmente proposto em:

> [Attention is All You Need](https://arxiv.org/abs/1706.03762) - [Vaswani, A. et al. *Conference on Neural Information Processing Systems (NeurIPS)*, 2017]

## Imports

In [1]:
import math
import torch
import torch.nn as nn

Alguns comentários utilizam a extensão *Better Comments* (disponível no editor VS Code) para destacar informações, trazer questionamentos e expressar mais sem poluir o código. Seu uso é recomendado para melhor aproveitamento do material.

# Modelo

##### Input Embedding

In [ ]:
class InputEmbedding(nn.Module):
    """
    TODO: Add docstring
    """

    def __init__(self, d_model: int, vocab_size: int):
        super().__init__()
        self.d_model = d_model
        self.vocab_size = vocab_size
        self.embedding = nn.Embedding(vocab_size, d_model)

    def forward(self, x):
        return self.embedding(x) * math.sqrt(self.d_model)

##### Positional Encoding

Positional Encoding será uma função calculada **apenas uma vez** por posição. As senóides garantem que não ocorrerá repetição, uma vez que alterna a frequência do sinal conforme a feature!

\begin{equation} \nonumber
    \textrm{PE}(\textrm{pos}, 2i) = \sin\Big(\frac{\textrm{pos}}{10000^{ \frac{2i}{d_{\textrm{model}}} }}\Big)
\end{equation}

\begin{equation} \nonumber
    \textrm{PE}(\textrm{pos}, 2i+1) = \cos\Big(\frac{\textrm{pos}}{10000^{ \frac{2i}{d_{\textrm{model}}} }}\Big)
\end{equation}

In [ ]:
class PositionalEncoding(nn.Module):
    """
    TODO: Verificar docstring
    Implementa a codificacao posicional do Transformer
    A codificacao posicional e uma matriz de dimensao (seq_len x d_model)
    onde cada linha representa a codificacao de uma palavra na sequencia
    A codificacao e feita utilizando funcoes trigonométricas, onde a posicao
    da palavra e representada por um vetor de dimensao (1 x d_model)
    """

    def __init__(self, d_model: int, seq_len: int, dropout: float):
        super().__init__()
        self.d_model = d_model
        self.seq_len = seq_len
        self.dropout = nn.Dropout(dropout)

        #* Matriz de dimensão (seq_len x d_model)
        pe = torch.zeros(seq_len, d_model)

        # Vetor de posição da palavra (seq_len x 1)
        pos = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1) #? torch.float16 -> mais eficiente?
        # Para maior estabilidade numerica utiliza-se o log 
        denominator = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        # Aplica trigonometricas
        pe[:, 0::2] = torch.sin(pos * denominator)
        pe[:, 1::2] = torch.cos(pos * denominator)

        #* Transforma PE em um tensor de dim (batch_size x seq_len x d_model)
        pe = pe.unsqueeze(0) # (1 x seq_len x d_model)
        # Garante que o tensor seja salvo junto com o estado do modelo
        self.register_buffer('pe', pe)

    def forward(self, x):
        # PE nao e aprendido, nao requer gradiente
        x += self.pe[:, :x.shape[1], :].requires_grad_(False)
        return self.dropout(x)

##### Normalização


Como a distribuição de inputs nas camadas mais profundas está sujeita a mudanças bruscas devido a aprendizados em camadas mais razas, a Normalização é necessária para garantir que, apesar de ainda ocorrer mudanças na distribuição pelos mesmo motivos, a média ($\mu$) e variância ($\sigma$) permenecerão as mesmas!

\begin{equation} \nonumber
    \mu = \frac{1}{|H|}\sum_{i \in H}{h_i} \quad , 
    \quad \sigma = \sqrt{\frac{1}{|H|} \sum_{i \in H}{(h_i - \mu)^2} }
\end{equation}

Normalizando:

\begin{equation} \nonumber
    \hat{h}_j = \frac{h_j - \mu}{\sigma_j + \epsilon} \quad \longrightarrow \quad \hat{h}_j =  \gamma \frac{h_j - \mu}{\sigma_j + \epsilon} + \delta
\end{equation}

Isso aumenta o tamanho do modelo, uma vez que a normalização ocorre a cada ativação numa camada, contudo, os benefícios observados são:
- **Estabilização da _Forward Propagation_**: se o offset \delta é inicializado com 0s e \gamma com 1s, então a variância irá aumentar de forma linear com as camadas. O aprendizado inicial será mais lento, porém a rede aprende \gamma e pode regular seu caminho efetivo
- **Taxa de aprendizado ($\eta$) maior**: Com a superfície de perda mais suave (e consequentemente seu gradiente) o gradiente também a mais dificilmente quebrado, assim, como a superfície é mais "previsível", pode-se utilizar taxas de aprendizado maiores 
- **Regularização**: Indiretamente, como a normalização de *batches* depende de estatísticas da própria leva (*batch*), o efeito prático é de pequenas variações nas ativações a cada iteração de treino, similar à introdução de ruído.

In [4]:
class LayerNormalization(nn.Module):
    """
    Epsilon (eps): parametro que evita divisao por 0 e estabiliza a normalizacao
    Gamma: fator multiplicativo  (aprendido)
    Delta: fator aditivo / offset (aprendido)
    """

    def __init__(self, eps: float = 10**-6):
        super().__init__()
        # Epsilon evita divisao por 0 e valores muito elevados na normalizacao
        self.eps = eps
        self.gamma = nn.Parameter(torch.ones(1))
        self.delta = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        """
        keep dimension: .mean() cancela a dimensao por padrao, keepdim evita isso
        """
        mean = x.mean(dim = -1, keepdim=True)
        std = x.std(dim = -1, keepdim=True)
        return self.gamma * (x - mean) / (std + self.eps) + self.delta 

##### Feed Forward

In [ ]:
class FeedForwardBlock(nn.Module):
    """
    Feed Forward Block:

    d_model -> d_ff -> d_model
    d_model: Dimensao de entrada e saida
    d_ff: Dimensao intermediaria
    dropout: taxa de dropout
    """

    def __init__(self, d_model: int, d_ff: int, dropout: float):
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff) # W1 e B1 (bias=True por padrao)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(d_ff, d_model) # W2 e B2
        self.layer_norm = LayerNormalization()

    def forward(self, x):
        # Feed Forward
        x = self.dropout(torch.relu(self.linear1(x))) #d_model -> d_ff
        return self.linear2(x) #d_ff -> d_model       

##### Multi-head Self Attention

In [ ]:
class MultiheadAttentionBlock(nn.Module):
    """
    Multihead Attention Block:
    d_model: Dimensao de entrada e saida
    n_heads: numero de cabecas
    dropout: taxa de dropout
    """

    def __init__(self, d_model: int, n_heads: int, dropout: float):
        super().__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        self.dropout = nn.Dropout(dropout)

        # d_model deve ser divisivel por n_heads
        assert d_model % n_heads == 0, f'd_model ({d_model}) must be divisible by n_heads ({n_heads})'
        self.head_dim = d_model // n_heads # d_k 

        # WQ, WK, WV -> (d_model x d_model)
        self.w_q = nn.Linear(d_model, d_model) # WQ e BQ (bias=True por padrao)
        self.w_k = nn.Linear(d_model, d_model) # WK e BK
        self.w_v = nn.Linear(d_model, d_model) # WV e BV

        # Linear de saida -> (d_model x d_model)
        self.w_out = nn.Linear(d_model, d_model) # WO e BO

    @staticmethod
    def attention(self, query, key, value, dropout: nn.Dropout, mask=None):
        """
        TODO: adicionar docstring
        Implementa a atencao do Transformer
        A atencao e feita utilizando a funcao softmax, onde a atencao e dada por:
        softmax(QK^T / sqrt(d_k))V

        Static method para permitir a chamada da funcao de atencao
        sem precisar instanciar a classe.
        """

        head_dim = query.shape[-1] # d_k

        # Multiplica Q e K transposto
        # (batch_size x n_heads x seq_len x head_dim) x (batch_size x n_heads x head_dim x seq_len)
        attention_score = torch.matmul(query, key.transpose(-2, -1)) # -> (batch_size x n_heads x seq_len x seq_len)
        attention_score /= math.sqrt(head_dim)

        if mask is not None:
            attention_score = attention_score.masked_fill(mask == 0, -1e9)
        attention_score = torch.softmax(attention_score, dim=-1) # softmax(QK^T / sqrt(d_k))

        if dropout is not None:
            attention_score = dropout(attention_score)

        # Aplicando a atencao no valor
        # (batch_size x n_heads x seq_len x seq_len) x (batch_size x n_heads x seq_len x head_dim)
        x = torch.matmul(attention_score, value) # -> (batch_size x n_heads x seq_len x head_dim)

        return x, attention_score

    def forward(self, q, k, v, mask=None):
        """"
        .contiguous() -> garante que o tensor esteja na memoria contigua, i.e., 
         habilita o uso de .view() e concatenacao direta das cabecas
        .transpose() -> troca a ordem das dimensoes do tensor
        .view() -> transforma o tensor em uma nova forma
        """
        # q, k, v: (batch_size x seq_len x d_model)
        batch_size = q.size

        query = self.w_q(q) # (batch_size x seq_len x d_model)
        key = self.w_k(k)
        value = self.w_v(v)
        
        # TODO: verificar se asserts estao corretos	(especialmente o de batch_size)
        assert query.size() == key.size() == value.size(), f'query ({query.size()}) != key ({key.size()}) != value ({value.size()})'
        assert query.size() == (batch_size, -1, self.d_model), f'query ({query.size()}) != (batch_size, -1, d_model)'

        # Transforma (batch_size x seq_len x d_model) em (batch_size x n_heads x seq_len x head_dim):

        # Transpose se faz para que a dimensao de cabeca fique na segunda posicao
        # e a dimensao de sequencia na terceira, de forma que cada cabeca veja todas as palavras
        # da sequencia
        query = query.view(query.shape[0], query.shape[1], self.n_heads, self.head_dim).transpose(1, 2)
        key = key.view(key.shape[0], key.shape[1], self.n_heads, self.head_dim).transpose(1, 2)
        value = value.view(value.shape[0], value.shape[1], self.n_heads, self.head_dim).transpose(1, 2)

        # query, key, value: (batch_size x n_heads x seq_len x head_dim)

        x, attention_score = MultiheadAttentionBlock.attention(self, query, key, value, self.dropout, mask)

        x = x.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model) # (batch_size x seq_len x d_model)
        return self.w_out(x)

##### Residual Connection (Skip Connection)

Responsável por gerenciar a passagem de outputs entre blocos

In [ ]:
class ResidualConnection(nn.Module):
    """
    Residual Connection:
    Implementa a conexao residual do Transformer
    A conexao residual e uma soma entre a entrada e a saida do bloco
    """

    def __init__(self, dropout: float):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = LayerNormalization()

    def forward(self, x, sublayer):
        return x + self.dropout(sublayer(self.layer_norm(x)))

### Encoder Block

\begin{equation} \nonumber
Encoder = N \times \Big(\textrm{Input} \longrightarrow \textrm{Multi-head SA} \rightarrow \textrm{Layer Norm} \longrightarrow \textrm{Feed Forward} \rightarrow \textrm{Layer Norm} \Big) 
\end{equation}

In [ ]:
class EncoderBlock(nn.Module):
    """
    Encoder Block:
    Implementa o bloco de codificacao do Transformer
    """

    def __init__(self, self_attention: MultiheadAttentionBlock, feed_forward: FeedForwardBlock, dropout: float):
        super().__init__()
        self.attention = self_attention
        self.feed_forward = feed_forward
        self.residual_connection = nn.ModuleList([
            ResidualConnection(dropout),
            ResidualConnection(dropout)
        ])

    def forward(self, x, src_mask=None):
        """
        TODO: completar docstring
        x: tensor de entrada (batch_size x seq_len x d_model)
        src_mask: mascara de entrada para evitar que 
        palavras de padding sejam consideradas na atencao
        """

        x = self.residual_connection[0](x, lambda x: self.attention(x, x, x, src_mask))
        x = self.residual_connection[1](x, self.feed_forward)
        return x


## Encoder

In [ ]:
class Encoder(nn.Module):
    """
    Encoder:
    Implementa o codificador do Transformer
    """

    def __init__(self, layers: nn.ModuleList):
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization()

    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x) # Normaliza a saida do Encoder

## Decoder

### Decoder Block

In [ ]:
class DecoderBlock(nn.Module):
    """
    Implementa o bloco de decoder

    """

    def __init__(
            self, 
            self_attention: MultiheadAttentionBlock, 
            cross_attention: MultiheadAttentionBlock, 
            feed_forward: FeedForwardBlock,
            dropout: float
            ):
        super().__init__()
        self.self_attention = self_attention
        self.cross_attention = cross_attention
        self.feed_forward = feed_forward
        self.residual_connections = nn.ModuleList([
            ResidualConnection(dropout),
            ResidualConnection(dropout),
            ResidualConnection(dropout)
        ])
    
    def forward(self, x, y, src_mask = None, trg_mask = None):
        """
        
        """

        x = self.residual_connections[0](x, lambda x: self.self_attention(x,x,x, trg_mask))
        x = self.residual_connections[1](x, lambda x: self.cross_attention(x, y, y, src_mask))
        x = self.residual_connections[2](x, self.feed_forward)

        return x

In [ ]:
class Decoder(nn.Module):
    """
    """

    def __init__(self, ):
        super().__init__()